In [1]:
import os
class Config:
    # 数据加载器配置
    DEBUG_MODE = True
    DEBUG_SAMPLE_SIZE = 5000
    BASE_DIR = os.getcwd()
    LIBRISPEECH_PATH = os.path.join(BASE_DIR, "devDataset", "LibriSpeech")
    SAMPLE_RATE = 16000
    DURATION = 1.0
    MAX_SAMPLES = int(SAMPLE_RATE * DURATION)
    NOISE_TYPES = ["white", "babble"]
    SNR_LEVELS = [0, 5, 10]
    NUM_WORKERS = 2
    VALID_RATIO = 0.1
    MAX_SEQ_LEN = 16000
    
    # 模型配置
    INPUT_DIM = 1
    HIDDEN_DIM = 256
    EMBEDDING_DIM = 128
    CHAOS_DIM = 64
    CHAOS_TIME_STEPS = 5
    ATTENTION_HEADS = 4
    
    # 训练配置
    EPOCHS = 500
    LR = 0.0008
    LR_DECAY = 0.95
    WEIGHT_DECAY = 1e-5
    SAVE_INTERVAL = 10
    VAL_INTERVAL = 1
    CHECKPOINT_DIR = "./checkpoints_T2"

    # 优化参数
    WARMUP_EPOCHS = 5
    GRAD_CLIP = 1.0

    # 损失函数权重
    CE_WEIGHT = 1.0
    ATTENTION_REG_WEIGHT = 0.01  # 注意力正则化权重（新增）

    # 早停参数
    PATIENCE = 15
    MIN_DELTA = 0.001

    # 内存优化参数
    GRADIENT_ACCUMULATION_STEPS = 4
    BATCH_SIZE = 8     # 减小批次大小，复杂注意力需要更多内存
    ENABLE_MIXED_PRECISION = False

    # 注意力机制特定参数
    ATTENTION_MAX_SEQ_LEN = 250  # 注意力机制处理的最大序列长度
    ATTENTION_HEAD_DIM = 64       # 注意力头维度
    ATTENTION_HEADS = 8           # 注意力头数量
    
    # 序列处理参数
    PADDING_MODE = 'constant'     # 填充模式
    PADDING_VALUE = 0.0           # 填充值
    
    BIFURCATION_THRESHOLD = 0.5  # 分岔阈值
    ATTENTION_DROPOUT = 0.1  # 注意力dropout率

    # 注意力监控参数
    ATTENTION_VIZ_INTERVAL = 5  # 注意力可视化间隔（epoch）
    ATTENTION_VIZ_DIR = "./attention_viz"  # 注意力可视化保存目录

In [2]:
import os
import random
import librosa
import numpy as np
import os
import random
import librosa
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import soundfile as sf
from tqdm import tqdm


# 噪声生成与注入
class NoiseInjector:
    @staticmethod
    def generate_white_noise(length):
        return np.random.randn(length).astype(np.float32)

    @staticmethod
    def generate_babble_noise(length, num_speakers=3):
        noise = np.zeros(length, dtype=np.float32)
        for _ in range(num_speakers):
            start = random.randint(0, max(0, length - Config.SAMPLE_RATE))
            end = min(start + Config.SAMPLE_RATE, length)
            noise[start:end] += np.random.randn(end - start).astype(np.float32)
        return noise / num_speakers

    @staticmethod
    def add_noise(signal, noise_type="white", snr_db=10):
        if len(signal) == 0:
            return signal

        signal_power = np.mean(signal ** 2)
        if signal_power < 1e-10:
            return signal

        signal_db = 10 * np.log10(signal_power)

        if noise_type == "white":
            noise = NoiseInjector.generate_white_noise(len(signal))
        elif noise_type == "babble":
            noise = NoiseInjector.generate_babble_noise(len(signal))
        else:
            raise ValueError(f"不支持的噪声类型：{noise_type}")

        noise_power = np.mean(noise ** 2)
        noise_db = -100 if noise_power < 1e-10 else 10 * np.log10(noise_power)

        target_noise_db = signal_db - snr_db
        noise_scale = 10 ** ((target_noise_db - noise_db) / 20)
        noisy_signal = signal + noise * noise_scale

        # 归一化
        max_val = np.max(np.abs(noisy_signal))
        if max_val > 1e-5:
            noisy_signal = noisy_signal / max_val

        return noisy_signal

# 数据集类
class SpeakerRecognitionDataset(Dataset):
    def __init__(self, split="train", add_noise=False, noise_type="white", snr_db=10):
        self.split = split
        self.add_noise = add_noise
        self.noise_type = noise_type
        self.snr_db = snr_db

        # 加载数据集
        self.audio_paths, self.labels = self._load_dataset()
        self.speaker_to_idx = self._build_speaker_map()

        # 调试模式
        if Config.DEBUG_MODE:
            if split == "train":
                self.audio_paths = self.audio_paths[:Config.DEBUG_SAMPLE_SIZE]
                self.labels = self.labels[:Config.DEBUG_SAMPLE_SIZE]
            elif split == "val":
                self.audio_paths = self.audio_paths[:min(Config.DEBUG_SAMPLE_SIZE // 2, len(self.audio_paths))]
                self.labels = self.labels[:min(Config.DEBUG_SAMPLE_SIZE // 2, len(self.labels))]

        # 验证数据集
        self._validate_dataset()
        print(f"最终 {split} 数据集大小: {len(self.audio_paths)} 个样本")

    def _load_dataset(self):
        audio_paths = []
        labels = []

        root = Config.LIBRISPEECH_PATH
        if not os.path.exists(root):
            print(f"错误: LibriSpeech路径不存在 - {root}")
            return [], []

        print(f"加载LibriSpeech数据集: {root}")

        # 只遍历 dev-clean 和 dev-other
        for subset_dir in ["dev-clean", "dev-other"]:
            subset_path = os.path.join(root, subset_dir)
            if not os.path.isdir(subset_path):
                continue

            print(f"处理子集: {subset_dir}")
            for speaker_dir in os.listdir(subset_path):
                speaker_path = os.path.join(subset_path, speaker_dir)
                if not os.path.isdir(speaker_path):
                    continue

                for chapter_dir in os.listdir(speaker_path):
                    chapter_path = os.path.join(speaker_path, chapter_dir)
                    if not os.path.isdir(chapter_path):
                        continue

                    for file in os.listdir(chapter_path):
                        if file.endswith(".flac"):
                            full_path = os.path.join(chapter_path, file)
                            audio_paths.append(full_path)
                            labels.append(speaker_dir)

            print(f"在 {subset_dir} 中找到 {len(audio_paths)} 个.flac文件")

        # 分割训练/验证
        if self.split != "test" and len(audio_paths) > 0:
            train_paths, val_paths, train_labels, val_labels = train_test_split(
                audio_paths, labels, test_size=Config.VALID_RATIO, random_state=42
            )
            if self.split == "train":
                return train_paths, train_labels
            else:
                return val_paths, val_labels

        return audio_paths, labels

    def _build_speaker_map(self):
        unique_speakers = sorted(set(self.labels))
        print(f"找到 {len(unique_speakers)} 个不同的说话人")
        return {speaker: idx for idx, speaker in enumerate(unique_speakers)}

    def _validate_dataset(self):
        if len(self.audio_paths) == 0:
            print(f"警告: {self.split}数据集为空")
            return

        valid_count = 0
        invalid_indices = []
        for i in range(len(self.audio_paths) - 1, -1, -1):
            path = self.audio_paths[i]
            try:
                if not os.path.exists(path):
                    raise FileNotFoundError("文件不存在")

                if os.path.getsize(path) < 1024:
                    raise ValueError("文件太小可能已损坏")

                if path.endswith('.flac'):
                    signal, sr = sf.read(path)
                else:
                    signal, sr = librosa.load(path, sr=Config.SAMPLE_RATE, mono=True)

                if len(signal) < Config.SAMPLE_RATE // 2:
                    raise ValueError("音频过短")

                if np.max(np.abs(signal)) < 1e-5:
                    raise ValueError("接近静音")

                valid_count += 1
            except Exception as e:
                invalid_indices.append(i)
                print(f"无效文件: {path} - {str(e)}")

        for i in invalid_indices:
            self.audio_paths.pop(i)
            self.labels.pop(i)

        print(f"有效文件: {valid_count}/{len(self.audio_paths) + len(invalid_indices)}")
        print(f"移除 {len(invalid_indices)} 个无效文件")
        self.speaker_to_idx = self._build_speaker_map()

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        path = self.audio_paths[idx]
        speaker_id = self.labels[idx]
        label = self.speaker_to_idx[speaker_id]

        try:
            if path.endswith('.flac'):
                signal, sr = sf.read(path)
                if sr != Config.SAMPLE_RATE:
                    signal = librosa.resample(signal, orig_sr=sr, target_sr=Config.SAMPLE_RATE)
            else:
                signal, sr = librosa.load(path, sr=Config.SAMPLE_RATE, mono=True)
        except Exception as e:
            print(f"加载音频错误 {path}: {str(e)}")
            signal = np.zeros(Config.MAX_SAMPLES, dtype=np.float32)

        # 确保音频长度正确
        if len(signal) > Config.MAX_SAMPLES:
            # 随机裁剪而不是固定裁剪开头
            start = random.randint(0, len(signal) - Config.MAX_SAMPLES)
            signal = signal[start:start + Config.MAX_SAMPLES]
        elif len(signal) < Config.MAX_SAMPLES:
            pad_len = Config.MAX_SAMPLES - len(signal)
            signal = np.pad(signal, (0, pad_len), 
                           mode=Config.PADDING_MODE, 
                           constant_values=Config.PADDING_VALUE)
        
        # 为注意力机制创建长度掩码
        seq_length = min(len(signal), Config.MAX_SAMPLES)
        attention_mask = np.ones(Config.MAX_SAMPLES, dtype=np.float32)
        if seq_length < Config.MAX_SAMPLES:
            attention_mask[seq_length:] = 0.0
        
        # 归一化
        max_val = np.max(np.abs(signal))
        if max_val > 1e-5:
            signal = signal / max_val
        
        # 添加噪声
        if self.add_noise and self.split == "train":
            noise_type = random.choice(Config.NOISE_TYPES)
            snr_db = random.choice(Config.SNR_LEVELS)
            signal = NoiseInjector.add_noise(signal, noise_type, snr_db)
        
        # 返回信号、标签和注意力掩码
        return torch.FloatTensor(signal), label, torch.FloatTensor(attention_mask)

# 数据加载器
def get_dataloaders(batch_size=None):
    if batch_size is None:
        batch_size = Config.BATCH_SIZE

    train_dataset = SpeakerRecognitionDataset(split="train")
    val_dataset = SpeakerRecognitionDataset(split="val")
    test_dataset = SpeakerRecognitionDataset(split="test")

    if len(train_dataset) == 0 and len(test_dataset) > 0:
        print("警告: 训练集为空，使用测试集作为训练集")
        train_dataset = test_dataset

    noisy_test_dataset = SpeakerRecognitionDataset(
        split="test", add_noise=True, noise_type="white", snr_db=5
    )

    print(f"训练集: {len(train_dataset)} 样本")
    print(f"验证集: {len(val_dataset)} 样本")
    print(f"测试集: {len(test_dataset)} 样本")
    print(f"带噪声测试集: {len(noisy_test_dataset)} 样本")
    print(f"总说话人数: {len(train_dataset.speaker_to_idx)}")

    # 创建自定义collate函数处理注意力掩码
    def collate_fn(batch):
        signals, labels, attention_masks = zip(*batch)
        signals = torch.stack(signals)
        labels = torch.tensor(labels)
        attention_masks = torch.stack(attention_masks)
        return signals, labels, attention_masks
    
    # 创建数据加载器时使用自定义collate函数
    dataloaders = {
        "train": DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                           num_workers=Config.NUM_WORKERS, pin_memory=True,
                           collate_fn=collate_fn),
        "val": DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=Config.NUM_WORKERS, pin_memory=True,
                         collate_fn=collate_fn),
        "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                          num_workers=Config.NUM_WORKERS, pin_memory=True,
                          collate_fn=collate_fn),
        "noisy_test": DataLoader(noisy_test_dataset, batch_size=batch_size, shuffle=False,
                                num_workers=Config.NUM_WORKERS, pin_memory=True,
                                collate_fn=collate_fn),
    }
    
    return dataloaders

if __name__ == "__main__":
    print("测试数据加载器...")
    dataloaders = get_dataloaders()
    x, y, attention_mask = next(iter(dataloaders["train"]))
    print(f"音频数据形状: {x.shape}")
    print(f"标签数据形状: {y.shape}")
    print(f"注意力掩码形状: {attention_mask.shape}")

测试数据加载器...
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 5000/5000
移除 0 个无效文件
找到 73 个不同的说话人
最终 train 数据集大小: 5000 个样本
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 557/557
移除 0 个无效文件
找到 73 个不同的说话人
最终 val 数据集大小: 557 个样本
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 5567/5567
移除 0 个无效文件
找到 73 个不同的说话人
最终 test 数据集大小: 5567 个样本
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 5567/5567
移除 0 个无效文件
找到 73 个不同的说话人
最终 test 数据集大小: 5567 个样本
训练集: 5000 样本
验证集: 557 样本
测试集: 5567 样本
带噪声测试集: 55

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# 简化混沌激励模块
class ChaoticStimulus(nn.Module):
    def __init__(self, input_dim, output_dim):
        """
        简化的混沌激励模块
        :param input_dim: 输入维度
        :param output_dim: 输出维度
        """
        super().__init__()
        self.chaos_transform = nn.Sequential(
            nn.Conv1d(input_dim, output_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(output_dim),
            nn.PReLU(),
            nn.Conv1d(output_dim, output_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(output_dim),
            nn.PReLU()
        )

        # 混沌扰动参数
        self.chaos_factor = nn.Parameter(torch.tensor(0.1))

    def forward(self, x):
        """
        前向传播
        :param x: 输入特征 [batch_size, channels, seq_len]
        :return: 混沌处理后的特征 [batch_size, channels, seq_len]
        """
        # 常规特征变换
        transformed = self.chaos_transform(x)

        # 添加混沌扰动
        batch_size, channels, seq_len = transformed.size()
        if self.training:  # 仅在训练时添加混沌扰动
            # 生成与特征相同形状的混沌噪声
            chaos_noise = torch.randn_like(transformed) * self.chaos_factor
            # 应用非线性激活增强混沌特性
            chaos_noise = torch.tanh(chaos_noise)
            transformed = transformed + chaos_noise

        return transformed


# 复杂分岔注意力机制
class BifurcationAttention(nn.Module):
    def __init__(self, input_dim, num_heads=Config.ATTENTION_HEADS,
                 threshold=Config.BIFURCATION_THRESHOLD):
        """
        分岔注意力机制
        :param input_dim: 输入维度
        :param num_heads: 注意力头数
        :param threshold: 分岔阈值
        """
        super().__init__()
        self.input_dim = input_dim
        self.num_heads = num_heads

        # 确保输入维度可以被头数整除
        assert input_dim % num_heads == 0, f"input_dim ({input_dim}) must be divisible by num_heads ({num_heads})"
        self.head_dim = input_dim // num_heads
        self.threshold = threshold

        # 注意力投影
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)

        # 输出投影
        self.out_proj = nn.Linear(input_dim, input_dim)

        # 分岔控制参数
        self.bifurcation_param = nn.Parameter(torch.tensor(0.5))

        # Dropout层
        self.dropout = nn.Dropout(Config.ATTENTION_DROPOUT)

        self.max_seq_len = Config.ATTENTION_MAX_SEQ_LEN

    def forward(self, x, attention_mask=None):
        """
        前向传播
        :param x: 输入特征 [batch_size, seq_len, input_dim]
        :param attention_mask: 注意力掩码 [batch_size, seq_len]
        :return: 注意力加权后的特征 [batch_size, seq_len, input_dim]
        """
        batch_size, seq_len, _ = x.size()

        # 限制序列长度防止内存溢出
        if seq_len > self.max_seq_len:
            x = x[:, :self.max_seq_len, :]
            seq_len = self.max_seq_len
            if attention_mask is not None:
                attention_mask = attention_mask[:, :self.max_seq_len]

        # 线性投影
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        # 分割头 [batch_size, num_heads, seq_len, head_dim]
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # 计算注意力分数
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)

        # 应用注意力掩码（如果提供）
        if attention_mask is not None:
            # 将注意力掩码扩展到注意力头的维度
            attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # [batch_size, 1, 1, seq_len]
            scores = scores.masked_fill(attention_mask == 0, -1e9)

        # 应用分岔控制
        # 分岔函数: f(r) = r * sin(π * r)
        r = torch.sigmoid(self.bifurcation_param)
        bifurcation_factor = r * torch.sin(np.pi * r)

        # 当分岔因子接近阈值时，系统动态变化加剧
        if torch.abs(bifurcation_factor - self.threshold) < 0.1:
            # 添加随机扰动模拟混沌行为
            scores = scores + 0.05 * torch.randn_like(scores, device=scores.device)

        # 应用softmax获取注意力权重
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        # 加权求和
        context = torch.matmul(attention_weights, v)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.input_dim)

        # 输出投影
        output = self.out_proj(context)

        return output, attention_weights


# 统计池化层
class StatisticalPooling(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        """
        前向传播
        :param x: 输入特征 [batch_size, channels, seq_len]
        :return: 池化后的特征 [batch_size, channels*2]
        """
        # 计算均值和标准差
        mean = torch.mean(x, dim=2)
        std = torch.std(x, dim=2)

        # 拼接均值和标准差
        return torch.cat((mean, std), dim=1)


# 完整的C-HiLAP模型（简化版+复杂注意力）
class CHiLAPModel(nn.Module):
    def __init__(self, input_dim=Config.INPUT_DIM, hidden_dim=Config.HIDDEN_DIM,
                 embedding_dim=Config.EMBEDDING_DIM, num_classes=None):
        """
        混沌层次吸引子传播(C-HiLAP)模型 - 简化版+复杂注意力
        """
        super().__init__()

        # 若未传入num_classes，可设置一个默认值（但实际使用时必须从数据集获取后传入）
        if num_classes is None:
            raise ValueError("必须指定num_classes（说话人数量），请从数据集获取后传入")

        # 特征提取层
        self.feature_extractor = nn.Sequential(
            nn.Conv1d(input_dim, hidden_dim, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(hidden_dim),
            nn.PReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(hidden_dim),
            nn.PReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(hidden_dim),
            nn.PReLU()
        )

        # 混沌激励模块
        self.chaos_layer = ChaoticStimulus(hidden_dim, hidden_dim)

        # 复杂注意力层
        self.attention = BifurcationAttention(hidden_dim)

        # TDNN层
        self.tdnn_block = nn.Sequential(
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, dilation=1, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.PReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, dilation=2, padding=2),
            nn.BatchNorm1d(hidden_dim),
            nn.PReLU()
        )

        # 池化层
        self.pooling = StatisticalPooling()

        # 嵌入层
        self.embedding = nn.Sequential(
            nn.Linear(hidden_dim * 2, embedding_dim),  # 统计池化输出channels*2
            nn.BatchNorm1d(embedding_dim),
            nn.PReLU(),
            nn.Dropout(0.2)
        )

        # 分类器
        self.classifier = nn.Linear(embedding_dim, num_classes)

    def forward(self, x, attention_mask=None):
        """
        前向传播
        :param x: 输入特征 [batch_size, channels, seq_len] 或 [batch_size, seq_len, channels]
        :param attention_mask: 注意力掩码 [batch_size, seq_len]
        :return: 嵌入向量和分类结果
        """
        # 检查输入维度并转换为正确的格式 [batch_size, channels, seq_len]
        if x.dim() == 3:
            # 如果是 [batch_size, seq_len, channels] 格式
            if x.size(1) > x.size(2):  # 序列长度应该大于通道数
                x = x.permute(0, 2, 1)  # 转换为 [batch_size, channels, seq_len]

        # 限制序列长度防止内存溢出
        seq_len = x.size(2)
        if seq_len > Config.MAX_SEQ_LEN:
            x = x[:, :, :Config.MAX_SEQ_LEN]
            if attention_mask is not None:
                attention_mask = attention_mask[:, :Config.MAX_SEQ_LEN]

        # 特征提取
        x = self.feature_extractor(x)

        # 混沌处理
        x = self.chaos_layer(x)

        # 转置维度以适应注意力模块 [batch_size, channels, seq_len] -> [batch_size, seq_len, channels]
        x = x.permute(0, 2, 1)

        # 调整注意力掩码的长度以匹配当前特征序列长度
        if attention_mask is not None:
            # 当前特征序列长度
            current_seq_len = x.size(1)
            # 原始注意力掩码的长度
            original_seq_len = attention_mask.size(1)
            if original_seq_len != current_seq_len:
                # 使用自适应平均池化1d来调整掩码长度
                attention_mask = attention_mask.unsqueeze(1)  # 添加通道维度
                attention_mask = F.adaptive_avg_pool1d(attention_mask, current_seq_len)
                attention_mask = attention_mask.squeeze(1)    # 移除通道维度

        # 注意力加权
        x, attention_weights = self.attention(x, attention_mask)

        # 转置回原始维度 [batch_size, seq_len, channels] -> [batch_size, channels, seq_len]
        x = x.permute(0, 2, 1)

        # TDNN处理
        x = self.tdnn_block(x)

        # 池化
        x = self.pooling(x)

        # 嵌入向量
        embedding = self.embedding(x)

        # 分类
        logits = self.classifier(embedding)

        return embedding, logits, attention_weights


# 注意力可视化工具
class AttentionVisualizer:
    def __init__(self):
        pass
    
    @staticmethod
    def plot_attention_weights(attention_weights, save_path=None):
        """
        绘制注意力权重热图
        :param attention_weights: 注意力权重 [batch_size, num_heads, seq_len, seq_len]
        :param save_path: 保存路径
        """
        try:
            import matplotlib.pyplot as plt
            import seaborn as sns
            
            # 取第一个样本和第一个注意力头
            attn = attention_weights[0, 0].detach().cpu().numpy()
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(attn, cmap='viridis')
            plt.title("Attention Weights Heatmap")
            plt.xlabel("Key Position")
            plt.ylabel("Query Position")
            
            if save_path:
                plt.savefig(save_path)
                print(f"注意力权重热图已保存到: {save_path}")
            else:
                plt.show()
                
        except ImportError:
            print("无法导入matplotlib或seaborn，跳过注意力可视化")
    
    @staticmethod
    def plot_bifurcation_parameter(model, save_path=None):
        """
        绘制分岔参数随时间的变化
        :param model: 模型实例
        :param save_path: 保存路径
        """
        try:
            import matplotlib.pyplot as plt
            
            # 获取分岔参数
            bifurcation_param = model.attention.bifurcation_param.detach().cpu().numpy()
            r = 1 / (1 + np.exp(-bifurcation_param))  # sigmoid逆变换
            bifurcation_factor = r * np.sin(np.pi * r)
            
            plt.figure(figsize=(10, 6))
            plt.plot([bifurcation_factor], 'ro-')
            plt.axhline(y=model.attention.threshold, color='r', linestyle='--', label='Threshold')
            plt.title("Bifurcation Parameter")
            plt.xlabel("Training Step")
            plt.ylabel("Bifurcation Factor")
            plt.legend()
            
            if save_path:
                plt.savefig(save_path)
                print(f"分岔参数图已保存到: {save_path}")
            else:
                plt.show()
                
        except ImportError:
            print("无法导入matplotlib，跳过分岔参数可视化")


# 测试代码
if __name__ == "__main__":
    # 创建模型实例
    model = CHiLAPModel(num_classes=10)  # 添加num_classes参数

    print("模型结构:")
    print(model)

    # 生成随机输入（使用较小的序列长度进行测试）
    batch_size = 2
    seq_len = Config.MAX_SEQ_LEN
    # 正确的输入格式：[batch_size, channels, seq_len]
    x = torch.randn(batch_size, 1, seq_len)
    seq_len_feat = seq_len // 8  # feature extractor stride=2 x3
    attention_mask = torch.ones(batch_size, seq_len_feat)

    print(f"\n测试前向传播:")
    print(f"输入形状: {x.shape}")
    print(f"注意力掩码形状: {attention_mask.shape}")

    # 前向传播
    try:
        embedding, logits, attention_weights = model(x, attention_mask)
        print(f"嵌入向量形状: {embedding.shape}")
        print(f"分类输出形状: {logits.shape}")
        print(f"注意力权重形状: {attention_weights.shape}")
        print("前向传播成功!")
        
        # 测试注意力可视化
        visualizer = AttentionVisualizer()
        visualizer.plot_attention_weights(attention_weights.unsqueeze(0), "attention_heatmap.png")
        visualizer.plot_bifurcation_parameter(model, "bifurcation_parameter.png")
        
    except Exception as e:
        print(f"前向传播错误: {e}")
        import traceback

        traceback.print_exc()

模型结构:
CHiLAPModel(
  (feature_extractor): Sequential(
    (0): Conv1d(1, 256, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=1)
    (3): Conv1d(256, 256, kernel_size=(5,), stride=(2,), padding=(2,))
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): PReLU(num_parameters=1)
    (6): Conv1d(256, 256, kernel_size=(5,), stride=(2,), padding=(2,))
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): PReLU(num_parameters=1)
  )
  (chaos_layer): ChaoticStimulus(
    (chaos_transform): Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
      (3): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): BatchNorm

Traceback (most recent call last):
  File "/run/nvme/job_5060361/tmp/ipykernel_4099380/1893251236.py", line 377, in <module>
    visualizer.plot_attention_weights(attention_weights.unsqueeze(0), "attention_heatmap.png")
  File "/run/nvme/job_5060361/tmp/ipykernel_4099380/1893251236.py", line 300, in plot_attention_weights
    sns.heatmap(attn, cmap='viridis')
  File "/usr/local/lib/python3.12/site-packages/seaborn/matrix.py", line 446, in heatmap
    plotter = _HeatMapper(data, vmin, vmax, cmap, center, robust, annot, fmt,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/seaborn/matrix.py", line 110, in __init__
    data = pd.DataFrame(plot_data)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib64/python3.12/site-packages/pandas/core/frame.py", line 827, in __init__
    mgr = ndarray_to_mgr(
          ^^^^^^^^^^^^^^^
  File "/usr/local/lib64/python3.12/site-packages/pandas/core/internals/construction.p

<Figure size 1000x800 with 0 Axes>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 训练器类
class Trainer:
    def __init__(self, config=Config, model=None):
        """初始化训练器"""
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"使用设备: {self.device}")

        # 创建模型
        if model is None:
            raise ValueError("初始化Trainer时必须传入model参数")
        self.model = model.to(self.device)  # 使用传入的模型

        # 定义损失函数
        self.ce_loss = nn.CrossEntropyLoss()

        # 定义优化器
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=config.LR,
            weight_decay=config.WEIGHT_DECAY
        )

        # 学习率调度器
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=5,
        )

        # 学习率预热调度器
        self.warmup_scheduler = optim.lr_scheduler.LambdaLR(
            self.optimizer,
            lr_lambda=lambda epoch: min(1.0, epoch / config.WARMUP_EPOCHS)
        )

        # 混合精度训练
        if config.ENABLE_MIXED_PRECISION and torch.cuda.is_available():
            self.scaler = torch.cuda.amp.GradScaler()
            print("启用混合精度训练")
        else:
            self.scaler = None

        # 创建检查点目录和注意力可视化目录
        os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)
        os.makedirs(config.ATTENTION_VIZ_DIR, exist_ok=True)

        # 早停计数器
        self.early_stop_counter = 0
        self.best_val_accuracy = 0.0
        print(f"模型预期输入长度: {config.MAX_SEQ_LEN}")

        # 训练历史记录
        self.train_history = {
            'loss': [],
            'accuracy': [],
            'val_loss': [],
            'val_accuracy': [],
            'attention_entropy': []  # 新增：注意力熵记录
        }

    def attention_regularization(self, attention_weights):
        """
        注意力正则化损失 - 鼓励注意力分布更加集中
        :param attention_weights: 注意力权重 [batch_size, num_heads, seq_len, seq_len]
        :return: 正则化损失
        """
        # 计算注意力分布的熵
        entropy = -torch.sum(attention_weights * torch.log(attention_weights + 1e-10), dim=-1)
        mean_entropy = torch.mean(entropy)
        
        # 记录注意力熵
        self.train_history['attention_entropy'].append(mean_entropy.item())
        
        return mean_entropy

    def train_one_epoch(self, dataloader, epoch):
        """
        训练一个epoch
        :param dataloader: 训练数据加载器
        :param epoch: 当前epoch
        :return: 平均训练损失和准确率
        """
        self.model.train()
        total_loss = 0.0
        total_ce_loss = 0.0
        total_attn_reg_loss = 0.0
        correct = 0
        total = 0

        # 梯度累积
        accumulation_steps = self.config.GRADIENT_ACCUMULATION_STEPS
        self.optimizer.zero_grad()

        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, (inputs, labels, attention_mask) in progress_bar:
            try:
                inputs, labels, attention_mask = inputs.to(self.device), labels.to(self.device), attention_mask.to(self.device)

                # 确保输入维度正确 [batch, 1, seq_len]
                if inputs.dim() == 2:  # [batch, seq_len]
                    inputs = inputs.unsqueeze(1)  # 添加通道维度 -> [batch, 1, seq_len]

                # 确保音频长度正确
                if inputs.size(2) > self.config.MAX_SEQ_LEN:
                    inputs = inputs[:, :, :self.config.MAX_SEQ_LEN]
                    attention_mask = attention_mask[:, :self.config.MAX_SEQ_LEN]
                elif inputs.size(2) < self.config.MAX_SEQ_LEN:
                    pad_len = self.config.MAX_SEQ_LEN - inputs.size(2)
                    inputs = torch.nn.functional.pad(inputs, (0, pad_len), value=0.0)
                    # 更新注意力掩码
                    attention_mask_padded = torch.ones(
                        inputs.size(0), self.config.MAX_SEQ_LEN, 
                        device=inputs.device
                    )
                    attention_mask_padded[:, :attention_mask.size(1)] = attention_mask
                    attention_mask = attention_mask_padded

                # 使用混合精度训练
                if self.scaler is not None:
                    with torch.cuda.amp.autocast():
                        # 前向传播 - 注意：复杂注意力模型返回三个值
                        embeddings, logits, attention_weights = self.model(inputs, attention_mask)

                        # 计算损失
                        ce = self.ce_loss(logits, labels)
                        attn_reg = self.attention_regularization(attention_weights)
                        loss = (self.config.CE_WEIGHT * ce + 
                               self.config.ATTENTION_REG_WEIGHT * attn_reg)

                        # 梯度累积
                        loss = loss / accumulation_steps

                    # 反向传播
                    self.scaler.scale(loss).backward()

                    # 梯度裁剪
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.GRAD_CLIP)

                    if (i + 1) % accumulation_steps == 0:
                        self.scaler.step(self.optimizer)  # 优化器步骤
                        self.scaler.update()
                        # 2. 再调用预热调度器
                        if epoch <= self.config.WARMUP_EPOCHS:
                            self.warmup_scheduler.step()
                        self.optimizer.zero_grad()

                else:
                    # 标准训练（不使用混合精度）
                    # 前向传播 - 注意：复杂注意力模型返回三个值
                    embeddings, logits, attention_weights = self.model(inputs, attention_mask)

                    # 计算损失
                    ce = self.ce_loss(logits, labels)
                    attn_reg = self.attention_regularization(attention_weights)
                    loss = (self.config.CE_WEIGHT * ce + 
                           self.config.ATTENTION_REG_WEIGHT * attn_reg)

                    # 梯度累积
                    loss = loss / accumulation_steps
                    loss.backward()

                    # 梯度裁剪
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.GRAD_CLIP)

                    if (i + 1) % accumulation_steps == 0:
                        # 1. 先更新参数
                        self.optimizer.step()
                        # 2. 再调用学习率预热调度器（仅在预热阶段）
                        if epoch <= self.config.WARMUP_EPOCHS:
                            self.warmup_scheduler.step()  # 移动到optimizer.step()之后
                        self.optimizer.zero_grad()

                # 统计
                total_loss += loss.item() * accumulation_steps
                total_ce_loss += ce.item()
                total_attn_reg_loss += attn_reg.item()
                _, predicted = logits.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # 更新进度条
                if i % 10 == 0:  # 减少更新频率
                    accuracy = 100. * correct / total
                    avg_loss = total_loss / (i + 1)
                    progress_bar.set_description(
                        f"Epoch {epoch}, Loss: {avg_loss:.4f}, Acc: {accuracy:.2f}%, "
                        f"CE: {total_ce_loss/(i+1):.4f}, AttnReg: {total_attn_reg_loss/(i+1):.4f}"
                    )

                # 手动垃圾回收
                if i % 50 == 0:
                    torch.cuda.empty_cache()
                    gc.collect()

            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"内存不足错误在批次 {i}: {e}")
                    torch.cuda.empty_cache()
                    gc.collect()
                    continue
                else:
                    raise e

        avg_loss = total_loss / len(dataloader)
        accuracy = 100. * correct / total
        return avg_loss, accuracy

    def validate(self, dataloader):
        """
        验证模型性能
        :param dataloader: 验证数据加载器
        :return: 验证损失和准确率
        """
        self.model.eval()
        total_loss = 0.0
        total_ce_loss = 0.0
        total_attn_reg_loss = 0.0
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []

        with torch.no_grad():
            for i, (inputs, labels, attention_mask) in enumerate(dataloader):
                try:
                    inputs, labels, attention_mask = inputs.to(self.device), labels.to(self.device), attention_mask.to(self.device)

                    # 确保输入维度正确 [batch, 1, seq_len]
                    if inputs.dim() == 2:  # [batch, seq_len]
                        inputs = inputs.unsqueeze(1)  # 添加通道维度 -> [batch, 1, seq_len]

                    # 确保音频长度正确
                    if inputs.size(2) > self.config.MAX_SEQ_LEN:
                        inputs = inputs[:, :, :self.config.MAX_SEQ_LEN]
                        attention_mask = attention_mask[:, :self.config.MAX_SEQ_LEN]
                    elif inputs.size(2) < self.config.MAX_SEQ_LEN:
                        pad_len = self.config.MAX_SEQ_LEN - inputs.size(2)
                        inputs = torch.nn.functional.pad(inputs, (0, pad_len), value=0.0)
                        # 更新注意力掩码
                        attention_mask_padded = torch.ones(inputs.size(0), self.config.MAX_SEQ_LEN, device=inputs.device)
                        attention_mask_padded[:, :attention_mask.size(1)] = attention_mask
                        attention_mask = attention_mask_padded

                    # 前向传播 - 注意：复杂注意力模型返回三个值
                    embeddings, logits, attention_weights = self.model(inputs, attention_mask)

                    # 计算损失
                    ce = self.ce_loss(logits, labels)
                    attn_reg = self.attention_regularization(attention_weights)
                    loss = (self.config.CE_WEIGHT * ce + 
                           self.config.ATTENTION_REG_WEIGHT * attn_reg)

                    total_loss += loss.item()
                    total_ce_loss += ce.item()
                    total_attn_reg_loss += attn_reg.item()

                    # 统计准确率
                    _, predicted = logits.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()
                    
                    # 收集所有预测和标签用于混淆矩阵
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())

                    # 内存管理
                    if i % 20 == 0:
                        torch.cuda.empty_cache()
                        gc.collect()

                except RuntimeError as e:
                    if "out of memory" in str(e):
                        print(f"验证时内存不足: {e}")
                        torch.cuda.empty_cache()
                        gc.collect()
                        continue
                    else:
                        raise e

        avg_loss = total_loss / len(dataloader)
        accuracy = 100. * correct / total
        
        # 计算混淆矩阵
        cm = confusion_matrix(all_labels, all_predictions)
        
        return avg_loss, accuracy, cm

    def visualize_attention(self, dataloader, epoch):
        """
        可视化注意力权重
        :param dataloader: 数据加载器
        :param epoch: 当前epoch
        """
        self.model.eval()
        
        with torch.no_grad():
            # 获取一个批次的数据
            inputs, labels, attention_mask = next(iter(dataloader))
            inputs, labels, attention_mask = inputs.to(self.device), labels.to(self.device), attention_mask.to(self.device)
            
            # 确保输入维度正确
            if inputs.dim() == 2:
                inputs = inputs.unsqueeze(1)
                
            # 前向传播获取注意力权重
            _, _, attention_weights = self.model(inputs, attention_mask)
            
            # 可视化注意力权重
            try:
                # 取第一个样本和第一个注意力头
                attn = attention_weights[0, 0].cpu().numpy()
                
                plt.figure(figsize=(10, 8))
                sns.heatmap(attn, cmap='viridis')
                plt.title(f"Attention Weights - Epoch {epoch}")
                plt.xlabel("Key Position")
                plt.ylabel("Query Position")
                
                # 保存图像
                save_path = os.path.join(self.config.ATTENTION_VIZ_DIR, f"attention_epoch_{epoch}.png")
                plt.savefig(save_path)
                plt.close()
                
                print(f"注意力权重可视化已保存到: {save_path}")
                
            except Exception as e:
                print(f"注意力可视化失败: {e}")

    def plot_training_history(self):
        """绘制训练历史"""
        try:
            # 创建子图
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            
            # 绘制损失曲线
            axes[0, 0].plot(self.train_history['loss'], label='Training Loss')
            axes[0, 0].plot(self.train_history['val_loss'], label='Validation Loss')
            axes[0, 0].set_title('Loss Curve')
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Loss')
            axes[0, 0].legend()
            axes[0, 0].grid(True)
            
            # 绘制准确率曲线
            axes[0, 1].plot(self.train_history['accuracy'], label='Training Accuracy')
            axes[0, 1].plot(self.train_history['val_accuracy'], label='Validation Accuracy')
            axes[0, 1].set_title('Accuracy Curve')
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Accuracy (%)')
            axes[0, 1].legend()
            axes[0, 1].grid(True)
            
            # 绘制注意力熵曲线
            axes[1, 0].plot(self.train_history['attention_entropy'])
            axes[1, 0].set_title('Attention Entropy')
            axes[1, 0].set_xlabel('Epoch')
            axes[1, 0].set_ylabel('Entropy')
            axes[1, 0].grid(True)
            
            # 留空用于其他可视化
            axes[1, 1].axis('off')
            
            plt.tight_layout()
            
            # 保存图像
            save_path = os.path.join(self.config.CHECKPOINT_DIR, "training_history.png")
            plt.savefig(save_path)
            plt.close()
            
            print(f"训练历史可视化已保存到: {save_path}")
            
        except Exception as e:
            print(f"训练历史可视化失败: {e}")

    def train(self, train_dataloader, val_dataloader):
        """
        完整训练流程
        :param train_dataloader: 训练数据加载器
        :param val_dataloader: 验证数据加载器
        """
        print("开始训练T2模型（简化模型+复杂注意力）...")
        print(f"训练集批次数: {len(train_dataloader)}, 验证集批次数: {len(val_dataloader)}")

        for epoch in range(1, self.config.EPOCHS + 1):
            try:
                # 应用学习率预热
                if epoch <= self.config.WARMUP_EPOCHS:
                    self.warmup_scheduler.step()

                # 训练一个epoch
                train_loss, train_acc = self.train_one_epoch(train_dataloader, epoch)
                self.train_history['loss'].append(train_loss)
                self.train_history['accuracy'].append(train_acc)
                
                print(f"Epoch {epoch}/{self.config.EPOCHS}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

                # 验证
                val_loss, val_acc, val_cm = self.validate(val_dataloader)
                self.train_history['val_loss'].append(val_loss)
                self.train_history['val_accuracy'].append(val_acc)
                
                print(f"Epoch {epoch}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

                # 更新学习率
                self.scheduler.step(val_loss)
                current_lr = self.optimizer.param_groups[0]['lr']
                print(f"当前学习率: {current_lr:.6f}")

                # 定期可视化注意力权重
                if epoch % self.config.ATTENTION_VIZ_INTERVAL == 0:
                    self.visualize_attention(val_dataloader, epoch)

                # 早停检查 (基于验证准确率)
                if val_acc > self.best_val_accuracy + self.config.MIN_DELTA:
                    self.best_val_accuracy = val_acc
                    self.early_stop_counter = 0
                    # 保存最佳模型
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'val_loss': val_loss,
                        'val_accuracy': val_acc,
                        'confusion_matrix': val_cm
                    }, os.path.join(self.config.CHECKPOINT_DIR, 'best_model.pth'))
                    print(f"保存最佳模型，验证准确率: {val_acc:.2f}%")
                else:
                    self.early_stop_counter += 1
                    if self.early_stop_counter >= self.config.PATIENCE:
                        print(f"早停于第 {epoch} 轮")
                        break

                # 定期保存模型
                if epoch % self.config.SAVE_INTERVAL == 0:
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'train_loss': train_loss,
                        'train_accuracy': train_acc,
                        'confusion_matrix': val_cm
                    }, os.path.join(self.config.CHECKPOINT_DIR, f'model_epoch_{epoch}.pth'))

                # 清理内存
                torch.cuda.empty_cache()
                gc.collect()

            except KeyboardInterrupt:
                print("训练被用户中断")
                break
            except Exception as e:
                print(f"训练错误: {e}")
                torch.cuda.empty_cache()
                gc.collect()
                continue

        # 训练完成后绘制训练历史
        self.plot_training_history()

    def load_checkpoint(self, checkpoint_path):
        """
        加载检查点
        :param checkpoint_path: 检查点路径
        """
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        if 'optimizer_state_dict' in checkpoint:
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint.get('epoch', 0)
        print(f"从第 {epoch} 轮加载检查点")
        return epoch


# 评估器类
class Evaluator:
    def __init__(self, model, config=Config):
        """初始化评估器"""
        self.model = model
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()

    def evaluate_accuracy(self, dataloader):
        """
        评估模型准确率
        :param dataloader: 数据加载器
        :return: 准确率
        """
        correct = 0
        total = 0

        with torch.no_grad():
            for i, (inputs, labels, attention_mask) in enumerate(dataloader):
                try:
                    inputs, labels, attention_mask = inputs.to(self.device), labels.to(self.device), attention_mask.to(self.device)

                    # 确保输入维度正确
                    if inputs.dim() == 2:  # [batch, seq_len]
                        inputs = inputs.unsqueeze(1)  # 添加通道维度 -> [batch, 1, seq_len]

                    # 截断过长的序列
                    if inputs.size(2) > self.config.MAX_SEQ_LEN:
                        inputs = inputs[:, :, :self.config.MAX_SEQ_LEN]
                        attention_mask = attention_mask[:, :self.config.MAX_SEQ_LEN]

                    # 前向传播 - 注意：复杂注意力模型返回三个值
                    _, logits, _ = self.model(inputs, attention_mask)
                    _, predicted = logits.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()

                    # 内存管理
                    if i % 20 == 0:
                        torch.cuda.empty_cache()
                        gc.collect()

                except RuntimeError as e:
                    if "out of memory" in str(e):
                        print(f"评估时内存不足: {e}")
                        torch.cuda.empty_cache()
                        gc.collect()
                        continue
                    else:
                        raise e

        accuracy = 100. * correct / total
        return accuracy

    def analyze_attention_patterns(self, dataloader, num_samples=5):
        """
        分析注意力模式
        :param dataloader: 数据加载器
        :param num_samples: 分析的样本数量
        :return: 注意力模式分析结果
        """
        self.model.eval()
        attention_patterns = []
        
        with torch.no_grad():
            for i, (inputs, labels, attention_mask) in enumerate(dataloader):
                if i >= num_samples:
                    break
                    
                inputs, labels, attention_mask = inputs.to(self.device), labels.to(self.device), attention_mask.to(self.device)
                
                # 确保输入维度正确
                if inputs.dim() == 2:
                    inputs = inputs.unsqueeze(1)
                    
                # 前向传播获取注意力权重
                _, _, attention_weights = self.model(inputs, attention_mask)
                
                # 分析注意力模式
                for j in range(attention_weights.size(0)):  # 遍历批次中的每个样本
                    sample_attention = attention_weights[j]
                    
                    # 计算注意力集中度（每个头的平均熵）
                    attention_entropy = -torch.sum(sample_attention * torch.log(sample_attention + 1e-10), dim=-1)
                    mean_entropy = torch.mean(attention_entropy).item()
                    
                    # 计算注意力对角线强度（关注局部模式的程度）
                    diag_strength = torch.mean(torch.diagonal(sample_attention, dim1=-2, dim2=-1)).item()
                    
                    attention_patterns.append({
                        'sample_id': i * self.config.BATCH_SIZE + j,
                        'label': labels[j].item(),
                        'mean_entropy': mean_entropy,
                        'diag_strength': diag_strength,
                        'attention_weights': sample_attention.cpu().numpy()
                    })
        
        return attention_patterns


# 内存友好的数据加载器创建函数
def create_small_dataloaders(dataset_name, batch_size=4):
    """创建小批次的数据加载器以节省内存"""
    try:
        train_dataset = SpeakerRecognitionDataset(split="train")
        val_dataset = SpeakerRecognitionDataset(split="val")
        test_dataset = SpeakerRecognitionDataset(split="test")

        return {
            "train": DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                num_workers=0, pin_memory=False),
            "val": DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                              num_workers=0, pin_memory=False),
            "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                               num_workers=0, pin_memory=False),
        }
    except Exception as e:
        print(f"数据加载器创建错误: {e}")
        return None


# 测试代码
if __name__ == "__main__":
    # 设置较小的批次大小
    batch_size = Config.BATCH_SIZE

    print("创建数据加载器...")
    try:
        # 使用内存友好的数据加载器
        dataloaders = create_small_dataloaders("librispeech", batch_size=batch_size)

        if dataloaders is None:
            print("无法创建数据加载器，退出")
            exit(1)

        print(f"训练集批次数: {len(dataloaders['train'])}")
        print(f"验证集批次数: {len(dataloaders['val'])}")
        print(f"测试集批次数: {len(dataloaders['test'])}")

        # 关键修改：从训练集中获取实际说话人数量（类别数）
        num_speakers = len(dataloaders["train"].dataset.speaker_to_idx)
        print(f"数据集中实际说话人数量（类别数）: {num_speakers}")

        # 创建训练器前，先初始化模型并传入正确的num_classes
        print("创建模型和训练器...")
        
        config = Config()
        # 1. 初始化模型（传入正确的num_classes）
        model = CHiLAPModel(num_classes=num_speakers)
        # 2. 将模型传入Trainer，确保优化器初始化时模型已存在
        trainer = Trainer(config=config, model=model)  # 传入model参数

        # 测试一个批次
        print("测试前向传播...")
        try:
            # 注意：现在数据加载器返回三个值（输入、标签、注意力掩码）
            x, y, mask = next(iter(dataloaders["train"]))
            print(f"原始输入形状: {x.shape}, 标签形状: {y.shape}, 掩码形状: {mask.shape}")

            # 处理输入维度
            if x.dim() == 2:  # [batch, seq_len]
                x = x.unsqueeze(1)  # 添加通道维度 -> [batch, 1, seq_len]

            # 确保音频长度正确
            if x.size(2) > Config.MAX_SEQ_LEN:
                x = x[:, :, :Config.MAX_SEQ_LEN]
                mask = mask[:, :Config.MAX_SEQ_LEN]
            elif x.size(2) < Config.MAX_SEQ_LEN:
                pad_len = Config.MAX_SEQ_LEN - x.size(2)
                x = torch.nn.functional.pad(x, (0, pad_len), value=0.0)
                # 更新注意力掩码
                mask_padded = torch.ones(x.size(0), Config.MAX_SEQ_LEN)
                mask_padded[:, :mask.size(1)] = mask
                mask = mask_padded

            print(f"处理后输入形状: {x.shape}, 掩码形状: {mask.shape}")

            x = x.to(trainer.device)
            y = y.to(trainer.device)
            mask = mask.to(trainer.device)

            with torch.no_grad():
                embeddings, logits, attention_weights = trainer.model(x, mask)
                print(f"嵌入形状: {embeddings.shape}, 输出形状: {logits.shape}, 注意力权重形状: {attention_weights.shape}")
                print("前向传播测试成功!")
        except Exception as e:
            print(f"前向传播测试失败: {e}")
            import traceback

            traceback.print_exc()
            exit(1)

        # 开始训练
        print("开始训练...")
        trainer.train(dataloaders["train"], dataloaders["val"])

        # 创建评估器
        print("创建评估器...")
        evaluator = Evaluator(trainer.model)

        # 评估模型
        print("评估模型...")
        test_accuracy = evaluator.evaluate_accuracy(dataloaders["test"])
        print(f"测试集准确率: {test_accuracy:.2f}%")
        
        # 分析注意力模式
        print("分析注意力模式...")
        attention_patterns = evaluator.analyze_attention_patterns(dataloaders["test"], num_samples=3)
        for pattern in attention_patterns:
            print(f"样本 {pattern['sample_id']} (标签: {pattern['label']}): "
                  f"平均熵={pattern['mean_entropy']:.4f}, 对角线强度={pattern['diag_strength']:.4f}")

    except Exception as e:
        print(f"程序执行错误: {e}")
        import traceback

        traceback.print_exc()

创建数据加载器...
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 5000/5000
移除 0 个无效文件
找到 73 个不同的说话人
最终 train 数据集大小: 5000 个样本
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 557/557
移除 0 个无效文件
找到 73 个不同的说话人
最终 val 数据集大小: 557 个样本
加载LibriSpeech数据集: /users/tianyuey/Project/devDataset/LibriSpeech
处理子集: dev-clean
在 dev-clean 中找到 2703 个.flac文件
处理子集: dev-other
在 dev-other 中找到 5567 个.flac文件
找到 73 个不同的说话人
有效文件: 5567/5567
移除 0 个无效文件
找到 73 个不同的说话人
最终 test 数据集大小: 5567 个样本
训练集批次数: 625
验证集批次数: 70
测试集批次数: 696
数据集中实际说话人数量（类别数）: 73
创建模型和训练器...
使用设备: cuda
模型预期输入长度: 16000
测试前向传播...
原始输入形状: torch.Size([8, 16000]), 标签形状: torch.Size([8]), 掩码形状: torch.Size([8, 16000])
处理后输入形状: torch.Size([8, 1, 16000]), 掩码形状: torch.Size([8, 16000])


/usr/local/lib64/python3.12/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


嵌入形状: torch.Size([8, 128]), 输出形状: torch.Size([8, 73]), 注意力权重形状: torch.Size([8, 8, 250, 250])
前向传播测试成功!
开始训练...
开始训练T2模型（简化模型+复杂注意力）...
训练集批次数: 625, 验证集批次数: 70


Epoch 1, Loss: 4.2342, Acc: 3.58%, CE: 4.1949, AttnReg: 3.9282: 100%|██████████| 625/625 [00:58<00:00, 10.76it/s]


Epoch 1/500, Train Loss: 4.2339, Train Acc: 3.58%
Epoch 1, Val Loss: 4.0890, Val Acc: 4.67%
当前学习率: 0.000800
保存最佳模型，验证准确率: 4.67%


...

Epoch 112/500, Train Loss: 1.6296, Train Acc: 55.80%
Epoch 112, Val Loss: 1.5091, Val Acc: 58.53%
当前学习率: 0.000025


Epoch 113, Loss: 1.6147, Acc: 55.90%, CE: 1.6067, AttnReg: 0.8010: 100%|██████████| 625/625 [00:58<00:00, 10.74it/s]


Epoch 113/500, Train Loss: 1.6148, Train Acc: 55.94%
Epoch 113, Val Loss: 1.2805, Val Acc: 64.09%
当前学习率: 0.000025


Epoch 114, Loss: 1.6136, Acc: 56.10%, CE: 1.6057, AttnReg: 0.7920: 100%|██████████| 625/625 [00:58<00:00, 10.74it/s]


Epoch 114/500, Train Loss: 1.6129, Train Acc: 56.18%
Epoch 114, Val Loss: 1.3716, Val Acc: 63.55%
当前学习率: 0.000025
早停于第 114 轮
训练历史可视化已保存到: ./checkpoints_T2/training_history.png
创建评估器...
评估模型...
测试集准确率: 64.83%
分析注意力模式...
样本 0 (标签: 51): 平均熵=0.7785, 对角线强度=0.0020
样本 1 (标签: 51): 平均熵=0.5801, 对角线强度=0.0016
样本 2 (标签: 51): 平均熵=0.4537, 对角线强度=0.0023
样本 3 (标签: 51): 平均熵=0.4665, 对角线强度=0.0023
样本 4 (标签: 51): 平均熵=3.0330, 对角线强度=0.0036
样本 5 (标签: 51): 平均熵=0.7846, 对角线强度=0.0021
样本 6 (标签: 51): 平均熵=0.7193, 对角线强度=0.0040
样本 7 (标签: 51): 平均熵=0.3430, 对角线强度=0.0037
样本 8 (标签: 51): 平均熵=0.3234, 对角线强度=0.0015
样本 9 (标签: 51): 平均熵=0.8505, 对角线强度=0.0026
样本 10 (标签: 51): 平均熵=0.4510, 对角线强度=0.0022
样本 11 (标签: 51): 平均熵=2.4415, 对角线强度=0.0033
样本 12 (标签: 51): 平均熵=0.5113, 对角线强度=0.0016
样本 13 (标签: 51): 平均熵=0.4706, 对角线强度=0.0026
样本 14 (标签: 51): 平均熵=0.4680, 对角线强度=0.0027
样本 15 (标签: 51): 平均熵=0.4532, 对角线强度=0.0044
样本 16 (标签: 51): 平均熵=0.9774, 对角线强度=0.0020
样本 17 (标签: 51): 平均熵=0.4740, 对角线强度=0.0026
样本 18 (标签: 51): 平均熵=0.2884, 对角线强度=0.0030
样本 19 (标签: 5